Before running the next cell please add a shortcut to the shared folder at the root of your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Learning_with_Density_Matrices_and_Random_Features-Supplementary_Material/qmc/tf')
import layers as layers
import models as models

/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.CrossProduct.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrix2Dist.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrixRegression.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [ ]:
import warnings
import numpy as np
from sklearn.kernel_approximation import RBFSampler

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import gzip

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

In [ ]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer

In [ ]:
# from functions.keras_wrapper_qmc import KerasClassifier

# from sklearn.model_selection import RandomizedSearchCV, KFold
# from sklearn.metrics import make_scorer

### example

In [ ]:
!gdown --id 1aFtA0kpR1ro7xj8QmwCr86xRMzbpjNkN

Downloading...
From: https://drive.google.com/uc?id=1aFtA0kpR1ro7xj8QmwCr86xRMzbpjNkN
To: /content/letter-recognition.data
100% 713k/713k [00:00<00:00, 104MB/s]


In [ ]:
def load_letters():
    letters = pd.read_csv("letter-recognition.data", header=None)
    #letters = pd.read_csv("letter-recognition.data", header=None)
    print(letters.iloc[:,1:].head())
    print(letters.iloc[:,1:].describe())
    #print(letters.iloc[:,1:].info())
    print(np.unique(letters.iloc[:,0]))

    vector = letters.values[:,1:]
    labels = letters.values[:,0]

    for index, i in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']):
        labels[labels==i] = index
        #print(index, " ",  i)

    labels=  np.array(labels, dtype=np.int32)


    X_train, X_test, y_train, y_test = train_test_split(vector, labels, test_size=0.3, random_state=42, stratify=labels)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = load_letters()

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16
0   2   8   3   5   1   8  13   0   6   6  10   8   0   8   0   8
1   5  12   3   7   2  10   5   5   4  13   3   9   2   8   4  10
2   4  11   6   8   6  10   6   2   6  10   3   7   3   7   3   9
3   7  11   6   6   3   5   9   4   6   4   4  10   6  10   2   8
4   2   1   3   1   1   8   6   6   6   6   5   9   1   7   5  10
                 1             2   ...            15           16
count  20000.000000  20000.000000  ...  20000.000000  20000.00000
mean       4.023550      7.035500  ...      3.691750      7.80120
std        1.913212      3.304555  ...      2.567073      1.61747
min        0.000000      0.000000  ...      0.000000      0.00000
25%        3.000000      5.000000  ...      2.000000      7.00000
50%        4.000000      7.000000  ...      3.000000      8.00000
75%        5.000000      9.000000  ...      5.000000      9.00000
max       15.000000     15.000000  ...     15.000000     15.00000

[8 rows x

In [ ]:
# Extracting a stratified samples such that we can do a dimensionality reduction
# percentage_for_stratified_samples = 0.95
# random_state = 42

# X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=percentage_for_stratified_samples, random_state=random_state, stratify=y_train)
# X_test, _, y_test, _ = train_test_split(X_test, y_test, test_size=percentage_for_stratified_samples, random_state=random_state, stratify=y_test)

In [ ]:
# Transforming each feature into a 0 to 1 range
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
gammas = [2**i for i in range(-10,10)]

In [ ]:
# Sanity check for shape of data
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)
print("y_train unique: ", np.unique(y_train))
print("y_test unique: ", np.unique(y_test))

X_train:  (14000, 16)
y_train:  (14000,)
X_test:  (6000, 16)
y_test:  (6000,)
y_train unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
y_test unique:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


In [ ]:
from sklearn.kernel_approximation import RBFSampler
rbf_feature = RBFSampler(gamma=8, n_components = 1000)
X_train_features = rbf_feature.fit_transform(X_train)
X_test_features = rbf_feature.transform(X_test)

from sklearn.svm import LinearSVC

model_lsvc = LinearSVC(C=2**5,tol=1e-05)
model_lsvc.fit(X_train_features, y_train)
print(model_lsvc.score(X_test_features, y_test))

0.9578333333333333


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
input_dim = X_train.shape[1]
num_classes = np.unique(y_train).shape[0]
gamma = 2**-5
component_dim = 1000
random_state=0
num_eig=0
batch_size=8
epochs = 10

In [ ]:
# Create models from qmkdc classifier

def create_model(input_dim, num_classes, component_dim=100, gamma=1, lr=0.01, decay=0.,
                  random_state=None, eig_percentage=0, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False):
    '''This is a model generating function so that we can search over neural net
    parameters and architecture'''

    num_eig = round(eig_percentage * component_dim)

    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay)

    if type_of_rff == 'rff':
        fm_x = layers.QFeatureMapRFF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)
    #else:
    #    fm_x = layers.QFeatureMapORF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)

    if initialize_with_rff:
        qmkdc = models.QMKDClassifier(fm_x=fm_x, dim_x=component_dim, num_classes=num_classes)
        qmkdc.compile()
        qmkdc.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=20)

    qmkdc1 = models.QMKDClassifierSGD(input_dim=input_dim, dim_x=component_dim, num_eig=num_eig,
                                      num_classes=num_classes, gamma=gamma, random_state=random_state, fm_x=fm_x)

    if fix_rff:
        qmkdc1.layers[0].trainable = False

    qmkdc1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    if initialize_with_rff:
        qmkdc1.set_rhos(qmkdc.get_rhos())

    # qmkdc1.fit(X_train, y_train_bin, epochs=epochs, batch_size=batch_size)

    return qmkdc1

In [ ]:
# Tensorboard 
%load_ext tensorboard
%tensorboard --logdir "/gdrive/My Drive/logs/"

In [ ]:
# Creation of the model without random search. It can be used to verify the initial performance of the model.
import datetime, os
logdir = os.path.join("/gdrive/My Drive/logs", "letters-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model_mnist = create_model(input_dim, num_classes, component_dim=1000, gamma=8, lr=1e-04, decay=0.,
                  random_state=None, eig_percentage=1, initialize_with_rff=True,
                  type_of_rff="rff", fix_rff=False)

y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes))
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
model_mnist.fit(X_train, y_train_bin.numpy(), epochs=50, batch_size=batch_size, validation_data=(X_test, y_test_bin.numpy()), 
            callbacks=[tensorboard_callback])

Epoch 1/50
1750/1750 [==============================] - 117s 65ms/step - loss: 1.1433 - accuracy: 0.8740 - val_loss: 0.8385 - val_accuracy: 0.8935
Epoch 2/50
1750/1750 [==============================] - 112s 64ms/step - loss: 0.7361 - accuracy: 0.9201 - val_loss: 0.6603 - val_accuracy: 0.9220
Epoch 3/50
1750/1750 [==============================] - 113s 64ms/step - loss: 0.5687 - accuracy: 0.9474 - val_loss: 0.5784 - val_accuracy: 0.9350
Epoch 4/50
1750/1750 [==============================] - 113s 65ms/step - loss: 0.5024 - accuracy: 0.9584 - val_loss: 0.5302 - val_accuracy: 0.9438
Epoch 5/50
1750/1750 [==============================] - 113s 65ms/step - loss: 0.4565 - accuracy: 0.9620 - val_loss: 0.5007 - val_accuracy: 0.9493
Epoch 6/50
1750/1750 [==============================] - 114s 65ms/step - loss: 0.4206 - accuracy: 0.9690 - val_loss: 0.4812 - val_accuracy: 0.9490
Epoch 7/50
1750/1750 [==============================] - 113s 65ms/step - loss: 0.4049 - accuracy: 0.9715 - val_loss: 0

In [ ]:
# Final validation test
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
out = model_mnist.predict(X_test)
accuracy_score(y_test, out.argmax(axis=1))

0.72

In [ ]:
# Creation of KerasClassifier for sending to random search
model = KerasClassifier(build_fn=create_model, epochs=1, batch_size=16, verbose=1)

In [ ]:
# component dimension
from scipy.stats import randint
#components_dimensions = randint(20,1500)
components_dimensions = [1000]

# gamma
#gammas = [2 ** i for i in range(-25, 25)]
gammas = [2 ** i for i in range(4, 6)]

# number of eigen values
eig_values = [1 / 10, 25 / 100, 50 / 100, 1]

# initialize with rff
#initialize_with_rff = [True, False]
initialize_with_rff = [True]

# type_of_rff_components
type_of_rff = ['rff', 'orf']

# fix_rff
fix_rff = [True]

# learning algorithm parameters
lr = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
decay = [1e-6, 1e-9, 0]

# dictionary summary
param_grid = dict(input_dim=[input_dim], num_classes=[num_classes], component_dim=components_dimensions, gamma=gammas,
                  lr=lr, decay=decay,
                  random_state=[None], eig_percentage=eig_values, initialize_with_rff=initialize_with_rff,
                  type_of_rff=type_of_rff, fix_rff=fix_rff
                  )

In [ ]:
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid,
                              verbose=20, n_iter=1, n_jobs=1)

In [ ]:
# grid_result = grid.fit(X_train, y_train)
y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes)).numpy()

In [ ]:
cv_results_df = None
n_experiments = 25

for i in range(n_experiments):
  grid_result = grid.fit(X_train, y_train_bin)

  if cv_results_df is None:
    cv_results_df = pd.DataFrame(grid_result.cv_results_)
  else:
    cv_results_df = pd.concat([cv_results_df, pd.DataFrame(grid_result.cv_results_)])

  cv_results_df.to_csv('/gdrive/My Drive/qmkdc letters random search.csv')

In [ ]:
experiments = pd.read_csv("/gdrive/My Drive/letters.csv")

In [ ]:
experiments.groupby(["experiment", "gamma", "initialize_rhos", "lr", "eig_percentage", "type_of_rff", "fix_rff", "sgd"]).agg({"mean","std", "count"}).reset_index()

experiment       gamma  ... current_experiment      
                                                ...                std count
0                          qmc_rff    0.007812  ...           1.000000     3
1                      qmc_rff_sgd    0.007812  ...           0.462125    60
2                      qmc_rff_sgd    0.015625  ...           0.000000    35
3                      qmc_rff_sgd    0.031250  ...           1.357922    59
4                      qmc_rff_sgd    0.062500  ...           1.441722   182
5                      qmc_rff_sgd    0.125000  ...           0.000000     9
6                      qmc_rff_sgd    0.250000  ...           1.104532   193
7                      qmc_rff_sgd    0.500000  ...           1.177863   154
8                      qmc_rff_sgd    2.000000  ...           0.498012    86
9                      qmc_rff_sgd    4.000000  ...           1.149185   150
10                     qmc_rff_sgd    8.000000  ...           1.132347    99
11                     qmc_rff_sgd   16.000000  ...           0.000000    48
12  qmc_rff_sgd_wo_initialize_rhos    0.007812  ...           1.417051   250
13  qmc_rff_sgd_wo_initialize_rhos    0.015625  ...           1.100333   192
14  qmc_rff_sgd_wo_initialize_rhos    0.031250  ...           1.411255   248
15  qmc_rff_sgd_wo_initialize_rhos    0.062500  ...           1.308120   213
16  qmc_rff_sgd_wo_initialize_rhos    0.125000  ...           0.835643   132
17  qmc_rff_sgd_wo_initialize_rhos    0.250000  ...           1.972800   105
18  qmc_rff_sgd_wo_initialize_rhos    0.500000  ...           1.443376   241
19  qmc_rff_sgd_wo_initialize_rhos    1.000000  ...           1.229419    39
20  qmc_rff_sgd_wo_initialize_rhos    2.000000  ...           1.661114   110
21  qmc_rff_sgd_wo_initialize_rhos    4.000000  ...           1.417051   250
22  qmc_rff_sgd_wo_initialize_rhos    8.000000  ...           1.695853   158
23  qmc_rff_sgd_wo_initialize_rhos   16.000000  ...           1.217882   110
24  qmc_rff_sgd_wo_initialize_rhos   32.000000  ...           1.337021   212
25  qmc_rff_sgd_wo_initialize_rhos   64.000000  ...           0.586009   105
26  qmc_rff_sgd_wo_initialize_rhos  128.000000  ...           0.998379   169

[27 rows x 29 columns]